## 套件引入

In [1]:
import bs4
import requests
import json

## 學期更改
* 更新資料改這邊就好

In [2]:
currentYear = 109
currentSemester = 2

## 爬蟲程式碼
* 感謝 110 級大學長的程式碼
* 小弟在此邊上註解

In [27]:
Sclass = ['AM', 'LI', 'GL', 'IN', 'CCD', 'FI', 'AE', 'AB', 'LS', 'CDA', 'SC', 'WL', 'AP', 'GR', 'CCM', 'TODO', 'EL', 'KH', 'CS', 'IFD', 'IM', 'SO', 'AC', 'CD', 'CHS', 'FL', 'CC', 'EE', 'DA', 'DAP', 'CE', 'CM', 'LA']

all_course = [] #總課表

for sclass in Sclass:
    data = '<tr><td+width=""33%"">開課學年：' + str(currentYear) + '　　開課學期：第' + str(currentSemester) + '學期</td><td+width=""33%"">開課部別：大學部</td><td+width=""34%"">開課系所：無</td></tr><tr><td+width=""33%"">開課班級：無</td><td+width=""33%"">授課教師：無</td><td+width=""34%"">上課時間：無</td></tr>'
    req = requests.post('https://course.nuk.edu.tw/QueryCourse/QueryResult.asp', data = {'Condition':data,'Flag': 1,'OpenYear': currentYear,'Helf': currentSemester,'Pclass': 'A','Sclass': sclass,'Yclass': '','SirName': '','Sirno': '','WeekDay': '','Card': '','Subject': '','Language': '','Pre_Cono': '','Coname': ''})
    req.encoding = 'big5'
    soup = bs4.BeautifulSoup(req.text,'html.parser')
    soup = soup.find_all('tr')

    # i 是要略過非課程的資料
    i = 0

    for ele in soup:
        if i<=3:
            i+=1
            continue
        tdSet = ele.find_all('td')

        #微學分奇怪的格式 (更改: 用倒數二列的 微學分 來判斷)
        if "微學分" in tdSet[-2].text:
            continue    # 略過囉

        tempCourse = {}    # 存取課程的詳細資料[id, dept, name, grade, class, teacher, period, category, credit, location]
        tempCourse['id'] = tdSet[0].text.upper() + tdSet[1].text.upper()
        tempCourse['dept'] = sclass
        tempCourse['name'] = tdSet[5].text
        tempCourse['grade'] = tdSet[3].text
        tempCourse['class'] = tdSet[4].text
        tempCourse['teacher'] = tdSet[12].decode_contents()[0:-5].replace('<br/>',',')

        # 課程時間處理 以 list 存放多個時段
        course_time = []
        tempCourseTime = []
        print(tdSet[5].text, tdSet[12].text, tdSet[13].text, tdSet[14].text, tdSet[15].text)
        for i in range(14,21):
            if tdSet[i].text != '\u3000':    # 編碼問題
                for day in tdSet[i].text.split(','):
                    if day == 'X':
                        course_time.append([i-13, 0])
                    elif day == 'Y':
                        course_time.append([i-13, 4.5])   # 中午顯示 4.5
                    else:
                        try:
                            course_time.append([i-13, int(day)])
                        except ValueError:
                            print("Convert to int error: {}".format(day))    # 印出錯誤格式
                            pass

        tempCourse['period'] = course_time
        # 必修以 True 表示, 選修以 False
        if tdSet[7].text == '必':
            tempCourse['category'] = True
        else:
            tempCourse['category'] = False
        tempCourse['credit'] = tdSet[6].text
        tempCourse['location'] = tdSet[13].text

        # 放入字典
        all_course.append(tempCourse)

    # JSON存檔
    with open('AllCourse.json','w',encoding='utf8') as f:
        f.write(json.dumps(all_course, ensure_ascii=False).encode("utf8",errors='ignore').decode("utf8",errors='ignore'))

泰 B01-204 　 　
　
 程式設計與實習（二） 張保榮 C01-200 　 2,3,4
if 微學分: False  禁止外加 限修
 體育(二)-資工系 王士仁 　 　 　
　
 資料結構 蘇家輝 C01-200 　 　
if 微學分: False  限修
 數位電路實驗 潘欣泰 C01-208 　 2,3,4
　
 工程數學 殷堂凱 B01-B106 　 　
　
 組合語言與系統程式 余亞儒 C01-200 　 　
if 微學分: False  限修
 java程式設計 余亞儒 C01-209 　 5,6,7
　
 視窗程式設計 蘇家輝 C01-209 2,3,4 　
　
 智慧財產權保護法制與實用 待公布 C01-B104 　 　
　
 專題研究(一) 殷堂凱 　 　 　
　
 作業系統 吳俊興 B01-105 2,3,4 　
if 微學分: False  學程
 軟體工程 林文揚 B01-204其他教室 6,7,8 　
　
 影像處理 殷堂凱 C01-200其他教室 　 5,6,7
　
 網路安全 陳建源 B01-204 　 　
　
 人工智慧 蔡毓娟 L01-107 　 　
　
 永續運算應用與實務 吳俊興 C01-209 　 　
　
 智慧財產權保護法制與實用 待公布 C01-B104 　 　
if 微學分: False  限修
 資訊專題講座(二) 蘇家輝 C01-114 　 　
　
 永續運算應用與實務 吳俊興 C01-209 　 　
if 微學分: False  學程
 物聯網 余亞儒郭錦福 C01-200 　 　
　
 嵌入式多核心系統與軟體 郭錦福 C01-209其他教室 　 2,3,4
　
 量子計算 陳建源 C01-114 　 　
　
 醫學影像處理 殷堂凱 C01-200其他教室 　 　
　
 醫療資訊學導論 林文揚潘欣泰殷堂凱 B01-108 2,3,4 　
　
 計算機結構 郭錦福 C01-B102 　 　
if 微學分: False  學程
 深度學習 張保榮 C01-209 6,7,8 　
　
 再生能源憑證制度：理論與實務 連興隆 C01-B102 2,3,4 　
　
 永續運算應用與實務 吳俊興 C01-209 　 　
if 微學分: False  限修學程
 未來思考 連興隆 L01-202 　 5,6
if 微學分